In [2]:
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from scipy.fftpack import fft
from scipy.ndimage.filters import gaussian_filter

from sklearn.metrics import f1_score
# import tensorflow_addons as ta

plt.style.use('seaborn')

### Import Data

In [3]:
from utils import data

In [22]:
x_train, y_train = data.load_training_data()

In [23]:
print(f'Shape of x_train: {x_train.shape}')
print(f'Shape of y_train: {y_train.shape}')

Shape of x_train: (45360, 512, 3)
Shape of y_train: (45360,)


### Preprocessing

In [28]:
# Fourier transform data
x_train_fft = np.abs(fft(x_train, n=len(x_train[0]), axis=1))
x_train_fft = x_train_fft[:, :1 + int((len(x_train_fft[0]) - 1) / 2)]

### Build LSTM Model

In [6]:
# Generate weights for training (imbalanced classes)
class_weight = {}

count_0, count_1, count_2 = np.bincount(y_train)
total = count_0 + count_1 + count_2

for i in range(3):
    class_weight[i] = (1 / [count_0, count_1, count_2][i]) * total

In [38]:
# Build model
model_lstm = keras.Sequential([
    keras.layers.Conv1D(filters=16, input_shape=(x_train_fft.shape[1], x_train_fft.shape[2]), kernel_size=(5), 
                        activation='tanh'), # , kernel_regularizer='l2', padding='same'),
    
    #keras.layers.LSTM(32, return_sequences=True),
    #keras.layers.Dropout(0.2),
    keras.layers.LSTM(32, return_sequences=False),
    # keras.layers.Dropout(0.2),
    
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(3, activation='softmax')
])

# Choose Adam optimizer & categorical crossentropy
model_lstm.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', 
                   metrics = ['accuracy'])

In [39]:
history_lstm = model_lstm.fit(x_train_fft, y_train, class_weight=class_weight, epochs=10)

Epoch 1/10
45360/45360 [==============================] - 241s 5ms/sample - loss: 1.4590 - acc: 0.8031
Epoch 2/10
45360/45360 [==============================] - 241s 5ms/sample - loss: 0.8656 - acc: 0.9159
Epoch 3/10
45360/45360 [==============================] - 238s 5ms/sample - loss: 0.7277 - acc: 0.9289
Epoch 4/10
45360/45360 [==============================] - 240s 5ms/sample - loss: 0.7094 - acc: 0.9281
Epoch 5/10
45360/45360 [==============================] - 271s 6ms/sample - loss: 0.6877 - acc: 0.9329
Epoch 6/10
45360/45360 [==============================] - 273s 6ms/sample - loss: 0.6594 - acc: 0.9341
Epoch 7/10
45360/45360 [==============================] - 260s 6ms/sample - loss: 0.6534 - acc: 0.9327s - loss: 0.65
Epoch 8/10
45360/45360 [==============================] - 263s 6ms/sample - loss: 0.6261 - acc: 0.9364
Epoch 9/10
45360/45360 [==============================] - 262s 6ms/sample - loss: 0.6170 - acc: 0.9384
Epoch 10/10
45360/45360 [==============================] - 

In [40]:
train_f1 = f1_score(y_train, np.argmax(model_lstm.predict(x_train_fft), axis=1), average='macro')
print(f'F1 score on training data: {train_f1:.3f}')

F1 score on training data: 0.890


### Evaluate on Test Data

In [ ]:
x_test = load_testing_data()

In [ ]:
# Get prediction & generate submission file
y_test_pred = model_lstm.predict(x_test)
data.generate_submission(y_test_pred, 'LSTM_Pred')